In [1]:
## columns to drop
# How can we use transaction number and date as categorical values?
# drop transaction number
# should we filter out transaction sub type column?
# is free hold? turn into dummy variables.
# filtering out usage as residential - elimiates 4263 data points.
# should we filter out land from the property type column?
# property sub type? it lists things like clinic
#drop property sub type column, keep property type
#dropping transaction size because there are blank values. keeping property size.
#how should we filter out the rooms column? should we create an "other" category out of all the subcategories that aren't listed: gym, hotel, shop
# use KNN method to predict room values 
# use get dummies for parking column
# drop nearest metro, mall, and landmark categories.

In [40]:
# Import the modules
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt
import seaborn as sns


In [41]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("resources/trimmed.csv")
df = pd.read_csv(file_path)
# Review the DataFrame
df.head()

,Transaction Date,Transaction Type,Transaction sub type,Registration type,Is Free Hold?,Usage,Area,Property Type,Amount,Property Size (sq.m),Room(s)
0,2023-01-02 06:56:17,Sales,Sale,Ready,Free Hold,Residential,ARJAN,Unit,1056207.41,94.05,1 B/R
1,2023-01-02 07:19:57,Sales,Sale,Ready,Non Free Hold,Residential,Al Wasl,Land,5000000.00,1021.93,NaN
2,2023-01-02 07:25:49,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,BUSINESS BAY,Unit,2631000.00,105.75,2 B/R
3,2023-01-02 07:30:03,Sales,Delayed Sell,Ready,Free Hold,Residential,Al Hebiah Fifth,Land,2039000.00,143.99,NaN
4,2023-01-02 07:35:30,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,SOBHA HEARTLAND,Unit,3126741.00,147.08,3 B/R


In [42]:
#remove Commercial properties from Usage column
df = df[df['Usage'] != 'Commercial']
df

,Transaction Date,Transaction Type,Transaction sub type,Registration type,Is Free Hold?,Usage,Area,Property Type,Amount,Property Size (sq.m),Room(s)
0,2023-01-02 06:56:17,Sales,Sale,Ready,Free Hold,Residential,ARJAN,Unit,1056207.41,94.05,1 B/R
1,2023-01-02 07:19:57,Sales,Sale,Ready,Non Free Hold,Residential,Al Wasl,Land,5000000.00,1021.93,NaN
2,2023-01-02 07:25:49,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,BUSINESS BAY,Unit,2631000.00,105.75,2 B/R
3,2023-01-02 07:30:03,Sales,Delayed Sell,Ready,Free Hold,Residential,Al Hebiah Fifth,Land,2039000.00,143.99,NaN
4,2023-01-02 07:35:30,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,SOBHA HEARTLAND,Unit,3126741.00,147.08,3 B/R
...,...,...,...,...,...,...,...,...,...,...,...
81596,2023-06-26 14:59:47,Sales,Sale,Ready,Free Hold,Residential,BURJ KHALIFA,Unit,1790000.00,75.07,1 B/R
81597,2023-06-26 15:02:54,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,DUBAI SPORTS CITY,Unit,229850.00,31.64,Studio
81598,2023-06-26 15:08:28,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,AL FURJAN,Unit,2250000.00,272.59,3 B/R
81599,2023-06-26 15:41:46,Mortgage,Mortgage Registration,Ready,Free Hold,Residential,DUBAI CREEK HARBOUR,Unit,1242438.00,106.71,2 B/R


In [43]:
# Drop null values in rooms column
df.dropna(subset=['Room(s)'])

room_counts = df['Room(s)'].value_counts()
room_counts

Room(s)
1 B/R          23521
2 B/R          16128
Studio         11222
3 B/R           9423
4 B/R           3391
Office          1751
Shop             705
5 B/R            329
PENTHOUSE         63
6 B/R             33
Single Room        8
GYM                3
Hotel              2
7 B/R              2
9 B/R              1
Name: count, dtype: int64

In [44]:
# Drop additional columns.
new_df = df.drop(columns=['Transaction Date', 'Transaction sub type', 'Registration type','Usage'])
new_df


,Transaction Type,Is Free Hold?,Area,Property Type,Amount,Property Size (sq.m),Room(s)
0,Sales,Free Hold,ARJAN,Unit,1056207.41,94.05,1 B/R
1,Sales,Non Free Hold,Al Wasl,Land,5000000.00,1021.93,NaN
2,Sales,Free Hold,BUSINESS BAY,Unit,2631000.00,105.75,2 B/R
3,Sales,Free Hold,Al Hebiah Fifth,Land,2039000.00,143.99,NaN
4,Sales,Free Hold,SOBHA HEARTLAND,Unit,3126741.00,147.08,3 B/R
...,...,...,...,...,...,...,...
81596,Sales,Free Hold,BURJ KHALIFA,Unit,1790000.00,75.07,1 B/R
81597,Sales,Free Hold,DUBAI SPORTS CITY,Unit,229850.00,31.64,Studio
81598,Sales,Free Hold,AL FURJAN,Unit,2250000.00,272.59,3 B/R
81599,Mortgage,Free Hold,DUBAI CREEK HARBOUR,Unit,1242438.00,106.71,2 B/R


In [45]:
# Categorize features depending on their datatype.
obj = (new_df.dtypes == 'object')
object_cols = list(obj[obj].index)
print("Categorical variables:",len(object_cols))
 
int_ = (new_df.dtypes == 'int')
num_cols = list(int_[int_].index)
print("Integer variables:",len(num_cols))
 
fl = (new_df.dtypes == 'float')
fl_cols = list(fl[fl].index)
print("Float variables:",len(fl_cols))
new_df

Categorical variables: 5
Integer variables: 0
Float variables: 2


,Transaction Type,Is Free Hold?,Area,Property Type,Amount,Property Size (sq.m),Room(s)
0,Sales,Free Hold,ARJAN,Unit,1056207.41,94.05,1 B/R
1,Sales,Non Free Hold,Al Wasl,Land,5000000.00,1021.93,NaN
2,Sales,Free Hold,BUSINESS BAY,Unit,2631000.00,105.75,2 B/R
3,Sales,Free Hold,Al Hebiah Fifth,Land,2039000.00,143.99,NaN
4,Sales,Free Hold,SOBHA HEARTLAND,Unit,3126741.00,147.08,3 B/R
...,...,...,...,...,...,...,...
81596,Sales,Free Hold,BURJ KHALIFA,Unit,1790000.00,75.07,1 B/R
81597,Sales,Free Hold,DUBAI SPORTS CITY,Unit,229850.00,31.64,Studio
81598,Sales,Free Hold,AL FURJAN,Unit,2250000.00,272.59,3 B/R
81599,Mortgage,Free Hold,DUBAI CREEK HARBOUR,Unit,1242438.00,106.71,2 B/R


In [46]:
# Create OTHER category in Rooms column for values that are not numerical.
other_rooms = ['GYM', 'Office', 'PENTHOUSE', 'Shop', 'Single Room', 'Hotel']
new_df.loc[new_df['Room(s)'].isin(other_rooms), 'Room(s)'] = 'Other'
new_df

,Transaction Type,Is Free Hold?,Area,Property Type,Amount,Property Size (sq.m),Room(s)
0,Sales,Free Hold,ARJAN,Unit,1056207.41,94.05,1 B/R
1,Sales,Non Free Hold,Al Wasl,Land,5000000.00,1021.93,NaN
2,Sales,Free Hold,BUSINESS BAY,Unit,2631000.00,105.75,2 B/R
3,Sales,Free Hold,Al Hebiah Fifth,Land,2039000.00,143.99,NaN
4,Sales,Free Hold,SOBHA HEARTLAND,Unit,3126741.00,147.08,3 B/R
...,...,...,...,...,...,...,...
81596,Sales,Free Hold,BURJ KHALIFA,Unit,1790000.00,75.07,1 B/R
81597,Sales,Free Hold,DUBAI SPORTS CITY,Unit,229850.00,31.64,Studio
81598,Sales,Free Hold,AL FURJAN,Unit,2250000.00,272.59,3 B/R
81599,Mortgage,Free Hold,DUBAI CREEK HARBOUR,Unit,1242438.00,106.71,2 B/R


In [47]:
# Replacing amount empty values with their mean values to make the data distribution symmetric.
new_df['Amount'] = new_df['Amount'].fillna(
  new_df['Amount'].mean())

new_dataset = df.dropna()
new_dataset.isnull().sum()


Transaction Date        0
Transaction Type        0
Transaction sub type    0
Registration type       0
Is Free Hold?           0
Usage                   0
Area                    0
Property Type           0
Amount                  0
Property Size (sq.m)    0
Room(s)                 0
dtype: int64

In [48]:
new_df.dtypes

Transaction Type         object
Is Free Hold?            object
Area                     object
Property Type            object
Amount                  float64
Property Size (sq.m)    float64
Room(s)                  object
dtype: object

In [49]:
#df["A"] = df["A"].astype("category")

category_columns = ['Transaction Type', 'Is Free Hold?', 'Area', 'Property Type', 'Room(s)']

# Convert selected columns to categorical data type
new_df[category_columns] = new_df[category_columns].astype('category')

new_df.dtypes

Transaction Type        category
Is Free Hold?           category
Area                    category
Property Type           category
Amount                   float64
Property Size (sq.m)     float64
Room(s)                 category
dtype: object

In [50]:
# Split our preprocessed data into our features and target arrays. y should be target, and X should be features.
y = new_df['Amount']

# The X variable should include all features except the target
X = new_df.drop(columns=['Amount'])

In [51]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [52]:
# Perform get_dummies on categorical data columns.
df_dummies = pd.get_dummies(new_df, columns=category_columns, drop_first=True)
df_dummies

,Amount,Property Size (sq.m),Transaction Type_Mortgage,Transaction Type_Sales,Is Free Hold?_Non Free Hold,Area_AL BARARI,Area_AL FURJAN,Area_AL KHAIL HEIGHTS,Area_AL WAHA,Area_ARABIAN RANCHES I,...,Property Type_Unit,Room(s)_2 B/R,Room(s)_3 B/R,Room(s)_4 B/R,Room(s)_5 B/R,Room(s)_6 B/R,Room(s)_7 B/R,Room(s)_9 B/R,Room(s)_Other,Room(s)_Studio
0,1056207.41,94.05,False,True,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1,5000000.00,1021.93,False,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2631000.00,105.75,False,True,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False
3,2039000.00,143.99,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,3126741.00,147.08,False,True,False,False,False,False,False,False,...,True,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81596,1790000.00,75.07,False,True,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
81597,229850.00,31.64,False,True,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
81598,2250000.00,272.59,False,True,False,False,True,False,False,False,...,True,False,True,False,False,False,False,False,False,False
81599,1242438.00,106.71,True,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False


In [54]:
# Split the data into features (X) and the target variable (y)
X = df_dummies.drop(columns=['Amount'])
y = df_dummies['Amount']

In [58]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [63]:
# Import relevant metrics - score, r2, mse, rmse, std - from Scikit-learn
from sklearn.metrics import mean_squared_error, r2_score
# Compute the metrics for the linear regression model
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

NameError: name 'model' is not defined

In [60]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error


In [61]:
from sklearn.ensemble import RandomForestRegressor
 
model_RFR = RandomForestRegressor(n_estimators=10)
model_RFR.fit(X_train, Y_train)
Y_pred = model_RFR.predict(X_valid)
 
mean_absolute_percentage_error(Y_valid, Y_pred)

ValueError: Found input variables with inconsistent numbers of samples: [61870, 65280]

In [62]:
from sklearn.linear_model import LinearRegression
 
model_LR = LinearRegression()
model_LR.fit(X_train, Y_train)
Y_pred = model_LR.predict(X_valid)
 
print(mean_absolute_percentage_error(Y_valid, Y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [61870, 65280]